In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [61]:
#Ideas

#How could user enter book: Enter part of title, then search in books dataframe with .title.str.contains(<input>), then sort results by number of ratings to get the most rated book and hence the highest chance of good recommendations

In [62]:
ratings = pd.read_csv('BX-Book-Ratings.csv', sep = ';' , encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore') #There are some suspicious ISBNs, we might loos them as they will not match with books on merging
ratings.rename(columns={'User-ID':'user'
                        , 'Book-Rating': 'rating'}, inplace=True)
ratings.shape
ratings.nunique()

(1149780, 3)

user      105283
ISBN      340553
rating        11
dtype: int64

In [63]:
# dtype_mapping = {
#     'Year-Of-Publication': float
# }
books = pd.read_csv('BX-Books.csv', sep = ';', encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore')

#books.drop(books[books['Unnamed: 8'].notna()].index, axis = 0, inplace=True) #Dropping the shifted rows. There is 19 of them, not worth the manual correction I think.
#books.drop(['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'], axis = 1, inplace=True)
books.rename(columns={'Book-Title':'title'
                        , 'Book-Author': 'author'
                        , 'Year-Of-Publication': 'year'
                        , 'Publisher': 'publisher'}, inplace=True)
books
books.nunique()

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
d:\Program_files\Anaconda_nw_install_2022\lib\site-packages\IPython\core\i

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


ISBN           271360
title          242135
author         102023
year              202
publisher       16807
Image-URL-S    271044
Image-URL-M    271044
Image-URL-L    271041
dtype: int64

In [64]:
users = pd.read_csv('BX-Users.csv', sep = ';' , encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore')
users.rename(columns={'User-ID':'user'
                        , 'Location': 'location'
                        , 'Age': 'age'}, inplace=True)
print(users.shape)
users.nunique()

(278858, 3)


user        278858
location     57293
age            165
dtype: int64

In [65]:
ratings

,user,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [66]:
#how many ratings each book have?
explicit_rating_counts = ratings[ratings['rating'] > 0][['ISBN', 'rating']].groupby('ISBN', as_index=False).count().sort_values('rating').rename(columns={'rating':'explicit_ratings'})
implicit_rating_counts = ratings[ratings['rating'] == 0][['ISBN', 'rating']].groupby('ISBN', as_index=False).count().sort_values('rating').rename(columns={'rating':'implicit_ratings'})
#implicit_rating_counts

if 'explicit_ratings' not in books.columns:
    books = books.merge(explicit_rating_counts, on = 'ISBN', how='left')
    books = books.merge(implicit_rating_counts, on = 'ISBN', how='left')

In [67]:
books.year.unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

In [68]:
#Clearing the years of publication
books.drop(books[books['year'] == 'Gallimard'].index, axis = 0, inplace=True)
books.drop(books[books['year'] == 'DK Publishing Inc'].index, axis = 0, inplace=True)
books['year'] = books['year'].astype(int, errors='ignore')

#books.loc[books.year == 0, 'year'] = np.NAN ## <<------- this forces the column to be floats. So I will let the zeros stay for now, see how it will affect the rest.

In [69]:
books.year.unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984,    0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, 2012, 2006, 1909, 2008, 1378,
       1919, 1922, 1897, 2024, 1376, 2037])

In [70]:
#books['year'] = books['year'].astype(int, errors='ignore')
books['year'].median()

1995.0

In [71]:
#Adding a time period when the book was released, instead of a year
#The idea here is to add primarily the tens of the book release, and then the most adjecent tens as a secondary variable
#Hence e.g. for 1991 -> 1990 and 1980, and for 1998 -> 1990 and 2000
books['period1'] = (books.year // 10) * 10
books['period2'] = books['period1'] + ((((books.year - books.period1) >= 5)*2-1)*10)


In [72]:
books[['ISBN', 'year', 'period1', 'period2']]

,ISBN,year,period1,period2
0,0195153448,2002,2000,1990
1,0002005018,2001,2000,1990
2,0060973129,1991,1990,1980
3,0374157065,1999,1990,2000
4,0393045218,1999,1990,2000
...,...,...,...,...
271355,0440400988,1988,1980,1990
271356,0525447644,1991,1990,1980
271357,006008667X,2004,2000,1990
271358,0192126040,1996,1990,2000


In [73]:
##Inputs/argumets

#books[(books['author'].str.contains('Tolkien')).replace(np.NaN, False)]
#books[(books['author'].str.contains('Robert Jordan')).replace(np.NaN, False)].head(10)
#books[(books['title'].str.contains('The Eye of the World')).replace(np.NaN, False)]
input_book_ISBN = '0812511816'
input_index = books[books.ISBN == input_book_ISBN].index.values[0]

choose_n = 5


In [74]:
books['info'] = ( books.title + ' ' +
                 (books.author.fillna('') + ' ')*1 
                 + books.publisher.fillna('') + ' ' 
                 + books.year.astype(str) + ' ' 
                 + books.period1.astype(str) + ' ' 
                 + books.period2.astype(str))
#Here possibly add weight by multiplying the number of times the info appears?
#But if I will use cosine similarity, this way of adding weights is probably useless (?): https://www.pinecone.io/learn/vector-similarity/

In [75]:
books[books['info'].isna()]
for i in books['info'][0:3]:
    print(i)

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L,explicit_ratings,implicit_ratings,period1,period2,info


Classical Mythology Mark P. O. Morford Oxford University Press 2002 2000 1990
Clara Callan Richard Bruce Wright HarperFlamingo Canada 2001 2000 1990
Decision in Normandy Carlo D'Este HarperPerennial 1991 1990 1980


In [76]:
vectorizer = CountVectorizer()
term_matrix = vectorizer.fit_transform(books['info'])

In [77]:
len(books)
term_matrix.shape
#print(term_matrix[0:5])

271357

(271357, 117284)

In [78]:
#similarities = cosine_similarity(term_matrix, dense_output=False)

In [79]:
#Computing similarities of all books takes too long. Perhaps I could select one book and compute similarity of it with the remaining.

In [80]:
row_to_compare = term_matrix[input_index, :]
similarities = cosine_similarity(row_to_compare, term_matrix)

In [82]:
similarities
similarities.shape

indices_of_largest = np.argsort(similarities)[-(choose_n+1):]
indi

array([[0.12171612, 0.12171612, 0.19245009, ..., 0.50037023, 0.18257419,
        0.29277002]])

(1, 271357)